### Calculo de media y varianza del dataset data_ok.csv
Septiembre 2024

v3: Transforma cada celda del dataset en un elemento (j,v),  donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

Realizado por:
- GEORGELYS MARCANO
- ERICH GONZÁLEZ
- DANIEL GUTIÉRREZ

Esta expresión para la **varianza** ofrece ventajas:

$$
\text{varianza} = \left(\frac{\sum x_i^2}{n}\right) - \left(\frac{\sum x_i}{n}\right)^2
$$

es **equivalente** a la fórmula estándar de la varianza.

$$
\text{varianza} = \frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2
$$

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import numpy as np
from pyspark import RDD

# Configuración de la sesión de Spark
session = SparkSession.builder.master('local[*]').getOrCreate()
context = session.sparkContext

FILE_PATH = 'data_ok.csv'
rdd0_raw = context.textFile(FILE_PATH)

def string_to_float(element):
    return np.array([float(x) for x in element.split()])
# Cada elemento del RDD es un numpy array de floats
rdd1_numpys = rdd0_raw.map(string_to_float)

def create_tuples(element):
    return [(key, value) for key, value in enumerate(element)]

rdd2 = rdd1_numpys.flatMap(create_tuples)

def prepare_for_calculations(rdd:RDD):
    def sums_and_counters(tuple):
        key,value = tuple
        return (key, (value, value**2, 1))
    
    return rdd.map(sums_and_counters)

rdd_aux = prepare_for_calculations(rdd2)

def calculate_mean(rdd:RDD) -> RDD:
    def reduce_function(value1, value2):
        val1, _, count1 = value1
        val2, _, count2 = value2
        return val1 + val2, 0, count1 + count2
    # Tener en cuenta que la estructura del retorno debe ser la misma que la del 
    # elemento de entrada para poder propagar el torneo, por lo que es necesario 
    # poner un valor para la suma de sqr, incluso si es cero.
    rdd_aux = rdd.reduceByKey(reduce_function)

    def map_function(element):
        key, (sum_values, _, counter) = element
        return (key, sum_values / counter)
    
    return rdd_aux.map(map_function)

rdd_mean = calculate_mean(rdd_aux)


def calculate_stdev(rdd:RDD) -> RDD:
    def reduce_function(value1, value2):
        val1, sqr1, count1 = value1
        val2, sqr2, count2 = value2
        return val1 + val2, sqr1 + sqr2, count1 + count2
    rdd_aux = rdd.reduceByKey(reduce_function)

    def map_function(element):
        key, (sum_val, sum_sqr, counter) = element
        variance = sum_sqr/counter - (sum_val/counter)**2
        return (key, variance**0.5)
    
    return rdd_aux.map(map_function)

rdd_stdev = calculate_stdev(rdd_aux)

print('Numero de elementos en rdd_mean', rdd_mean.count())
print('Numero de elementos en rdd_stdev', rdd_stdev.count())

# Es necesario ordenarlos por clave para poder cotejar que las tres
# versiones obtienen el mismo resultado.
#rdd_mean.sortByKey().take(5)
#rdd_stdev.sortByKey().take(5)
